In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/translation-m2m-dataset/DB.xlsx


In [2]:
!pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.4 MB/s eta 0:00:00


In [3]:
import wandb

wandb.login(key="b7fbf19e34af4a6f2b90b6b17e088e7a7a1c832b")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import pandas as pd                                      
import numpy as np
import matplotlib.pyplot as plt                                 
import re                                                  
import sacrebleu

#from sklearn.metrics import classification_report, accuracy_score,precision_score,f1_score,recall_score
from sklearn.model_selection import train_test_split

In [5]:
df=pd.read_excel("/kaggle/input/translation-m2m-dataset/DB.xlsx")


In [6]:
#df['egyption_Text'].values[4776]

In [7]:
#df['egyption_Text'].values[7459]

In [8]:
df=df[['english_Text','egyption_Text']]
df.head()

,english_Text,egyption_Text
0,"Sinbad, consider this matter resolved.","يا ""سندباد""، اعتبر الموضوع دا خلصان."
1,"Just send me the paper, and\nI will finish eve...",انت بس ابعتلي الولا بالورق،\nوأنا هأخلّصلك كل ...
2,OK dear. God protect you.,ماشي يا حبيبي. في رعاية الله.
3,"Greetings to Alaadin!Goodbye now.Good morning,...","سلامي لـ""الشاطر (حسن)""، هه!يلّا يا حبيبي، في ر..."
4,It's a morning\nlike any other morning.-Tell m...,والله صباح زي أي صباح!اتفضل يا سيدي، اؤمر!عندي...


In [9]:

import re
import string


def clean_text(text):
    # Replace newline characters with white space
    text = text.replace('\n', '  ')
    text = text.replace('.', '  ')
    # Remove any URL links
    text = re.sub(r'http\S+', '', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def clean_dataframe(df):
    cleaned_data = []
    for index, row in df.iterrows():
        # Clean English text
        clean_english = clean_text(row['english_Text'])
        
        # Clean Arabic text
        clean_arabic = clean_text(row['egyption_Text'])
        
        cleaned_data.append((clean_english, clean_arabic))
    
    cleaned_df = pd.DataFrame(cleaned_data, columns=['english_Text', 'egyption_Text'])
    
    return cleaned_df

# Example usage:
# Assuming your DataFrame is named 'df' and contains columns 'English' and 'Arabic'
# cleaned_df = clean_dataframe(df)


In [10]:
df_new=clean_dataframe(df)
df_new.head()

,english_Text,egyption_Text
0,sinbad consider this matter resolved,يا سندباد، اعتبر الموضوع دا خلصان
1,just send me the paper and i will finish every...,انت بس ابعتلي الولا بالورق، وأنا هأخلّصلك كل ح...
2,ok dear god protect you,ماشي يا حبيبي في رعاية الله
3,greetings to alaadingoodbye now good morning m...,سلامي لـالشاطر حسن، ههيلّا يا حبيبي، في رعاية ...
4,its a morning like any other morning tell me y...,والله صباح زي أي صباحاتفضل يا سيدي، اؤمرعندي أ...


In [11]:
def find_largest_sentence(series):
    max_length = 0
    max_sentence = ""
    max_index = -1

    for idx, sentence in enumerate(series):
        if isinstance(sentence, str):  # Check if the value is a string
            words = sentence.split()
            length = len(words)
            if length > max_length:
                max_length = length
                max_sentence = sentence
                max_index = idx

    return max_sentence, max_length, max_index

largest_sentence, word_count, max_index = find_largest_sentence(df_new['egyption_Text'])
print("Largest Sentence:", largest_sentence)
print("Number of Words:", word_count)
print("Index of the row:", max_index)


Largest Sentence: أعزائي المشاهدين، السلام عليكم ورحمة الله وبركاته،أهلًا بكم في حلقة جديدة، من برنامج الدحّيحعزيزي المشاهد الجميل، عايزك تتخيل للحظة أنك طائر نورس،بتُحلّق فوق البحر المتوسط متجه ناحية الشرق،بتوصل لنقطة على الساحل بين تل أبيب وحيفا،هناك، بيقابلك مكان رائع الجمال، يُعرف النهاردة باسم الدور،دا مَصيَف مثالي،لا يقِلّ حاجة عن شواطئ اليونان وقبرص وصقلية،رملة حلوة، جو لطيف، شُعَب مرجانية،وورا الشط مساحات خضرا مزروعة،ولو بتحب التاريخ والجغرافيا،فيه متحف، ومواقع أثرية من العصر البرونزي، وبيوت كتير،بيوت لمّا تشوفها، تحس إن البيوت دي جت من فلوريدا واتحطت هنا،اللي هي البيوت اللي عاملة كدا دي وطبعًا، فيه أكتر من resort في المكان مليانين خدمات،وبما إنك لسة، يا عزيزي، فاكر نفسك طائر نورس،ما يعرفش حاجة عن المكان،فهتشوف وانت فوق في السما كدا resort حديث،اسم هذا الـresort نحشوليم،بس على مسافة أمتار منه،فيه مبنى صغير، شكله وطرازه مختلف عن المكان،وبيكسر حالة التناسق بتاعة المكان،بيت مبني بـstyle عربي قديم وسط مباني حديثة وغربية خلّيني أقولّك إن سنة 1991،واحد من سكان المكان قرر إنه يبني جنب

In [12]:
def find_largest_sentence(series):
    max_length = 0
    max_sentence = ""
    max_index = -1

    for idx, sentence in enumerate(series):
        if isinstance(sentence, str):  # Check if the value is a string
            words = sentence.split()
            length = len(words)
            if length > max_length:
                max_length = length
                max_sentence = sentence
                max_index = idx

    return max_sentence, max_length, max_index

largest_sentence, word_count, max_index = find_largest_sentence(df_new['english_Text'])
'''print("Largest Sentence:", largest_sentence)
print("Number of Words:", word_count)
print("Index of the row:", max_index)
'''

'print("Largest Sentence:", largest_sentence)\nprint("Number of Words:", word_count)\nprint("Index of the row:", max_index)\n'

In [12]:



def count_words(text):
    # Split text into words
    words = re.findall(r'\b\w+\b', text)
    
    # Return the number of words
    return len(words)

def count_words_in_dataframe(df):
    word_counts = {}
    for column in df.columns:
        word_counts[column] = []
        for index, row in df.iterrows():
            word_count = count_words(row[column])
            word_counts[column].append(word_count)
    return word_counts

# Example usage:
# Assuming your DataFrame is named 'df'
# word_counts = count_words_in_dataframe(df)
# word_counts will be a dictionary where keys are column names and values are lists of word counts for each row in that column


In [13]:
l=count_words_in_dataframe(df_new)

In [14]:
english_words=l['english_Text']
egyption_words=l['egyption_Text']

In [15]:
english_words=sorted(english_words)
egyption_words=sorted(egyption_words)

In [16]:
english_words[-1 ]

5373

In [17]:
for i in range(20):
    print(egyption_words[-i])

1
5007
2088
2054
2003
1568
1405
1346
1128
1078
1052
992
946
872
674
669
665
640
636
632


In [20]:
x=0
for i in range(len(egyption_words)):
    if egyption_words[i]>50:
        x+=1
        
print(x)

61


In [18]:
for i in range(20):
    print(english_words[-i])

1
5373
2726
2407
2292
1744
1331
1299
1293
1152
1101
1073
1063
891
884
712
707
683
677
653


In [21]:
y=0
for i in range(len(egyption_words)):
    if english_words[i]>50:
        y+=1
        
print(y)

83


In [20]:
'''
def split_long_sentences(df, max_words=128):
    new_rows = []
    for index, row in df.iterrows():
        arabic_sentence = row['egyption_Text']
        english_sentence = row['english_Text']
        
        # Splitting sentences into words
        arabic_words = arabic_sentence.split()
        english_words = english_sentence.split()
        
        # Check if the sentence length exceeds the threshold
        if len(arabic_words) > max_words or len(english_words) > max_words:
            # Calculate the number of new rows needed
            num_splits = max(len(arabic_words), len(english_words)) // max_words + 1
            
            # Split the sentences into chunks of max_words
            for i in range(num_splits):
                start_idx = i * max_words
                end_idx = (i + 1) * max_words
                new_arabic = ' '.join(arabic_words[start_idx:end_idx])
                new_english = ' '.join(english_words[start_idx:end_idx])
                new_rows.append({'egyption_Text': new_arabic, 'english_Text': new_english})
        else:
            new_rows.append({'egyption_Text': arabic_sentence, 'english_Text': english_sentence})
    
    # Create a new DataFrame with the split sentences
    new_df = pd.DataFrame(new_rows)
    return new_df

# Example usage:
# Assuming df is your pandas DataFrame with Arabic and English columns
# df = pd.read_csv('your_dataset.csv')
# new_df = split_long_sentences(df)
'''

In [21]:
#df_new=split_long_sentences(df_new)

In [22]:
df_new['english_Text'].values[4776]

'the dome was an industrial exhibition building which was made of concrete at the moment of the bombing its location was exactly under the center of the bomb'

In [23]:
df_new['egyption_Text'].values[4776]

'القبة كانت مبنى لمعرض صناعي، واللي كانت معمولة من الخرسانة،ولحظة القصف، مكانها كان تحت مركز القنبلة بالظبط،'

In [24]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9001 entries, 0 to 9000
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   egyption_Text  9001 non-null   object
 1   english_Text   9001 non-null   object
dtypes: object(2)
memory usage: 140.8+ KB


In [21]:
'''
largest_sentence, word_count, max_index = find_largest_sentence(df_new['english_Text'])
print("Largest Sentence:", largest_sentence)
print("Number of Words:", word_count)
print("Index of the row:", max_index)
'''

'\nlargest_sentence, word_count, max_index = find_largest_sentence(df_new[\'english_Text\'])\nprint("Largest Sentence:", largest_sentence)\nprint("Number of Words:", word_count)\nprint("Index of the row:", max_index)\n'

In [26]:
'''
largest_sentence, word_count, max_index = find_largest_sentence(df_new['egyption_Text'])
print("Largest Sentence:", largest_sentence)
print("Number of Words:", word_count)
print("Index of the row:", max_index)
'''

'\nlargest_sentence, word_count, max_index = find_largest_sentence(df_new[\'egyption_Text\'])\nprint("Largest Sentence:", largest_sentence)\nprint("Number of Words:", word_count)\nprint("Index of the row:", max_index)\n'

In [21]:
df_new.shape,df.shape

((8744, 2), (8744, 2))

In [20]:
arabic_sentences = df_new['egyption_Text']
english_sentences = df_new['english_Text']

arabic_train, arabic_eval, english_train, english_eval = train_test_split(arabic_sentences, english_sentences, test_size=0.1, random_state=40)

In [22]:
arabic_eval.shape,english_eval.shape

((875,), (875,))

In [23]:
arabic_eval.iloc[0:10]

3132    أسطورة ألمانية مُلهِمة لأجيال كتير من الألمان،...
1556    وانت يا عم حسن؟ عايز لك كام صرّة؟حسن قال له، ص...
3071    وأحيانًا، التخلص منه، هو اللي بيخلّي عندنا حاض...
8435                                متخيل؟ وانت في الفضاء
4540    باختصار، يا عزيزي، الشخص الوحيد في اليابان الل...
1146    أول قرار خده عبد القادر مش إنه يحارب الفرنسيين...
2437    بلاكبيري دا، يا عزيزي، عبارة عن زراير وطلعلها ...
7951                          لأ، دا يمدهاله عشان يبوسها،
3437                     بس بيبقى فيها تفاؤل أكتر وانتصار
4924    رموز بصرية، مش محتاجة أي لغة، وكمان قادرة تحقق...
Name: egyption_Text, dtype: object

In [24]:
arabic_test=arabic_eval.iloc[0:100]

In [25]:
english_test=english_eval.iloc[0:100]

In [26]:
arabic_eval=arabic_eval.iloc[100:]

In [27]:
english_eval=english_eval.iloc[100:]

In [28]:
arabic_eval.shape,english_eval.shape

((775,), (775,))

In [29]:
english_test.iloc[79]

'i no longer like mark to be honest so i still want to boycott all of his products'

# Model

In [30]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM

from datasets import Dataset
import sacrebleu


model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name,src_lang="ar", tgt_lang="en")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M")

2024-05-05 18:17:00.225498: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 18:17:00.225631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 18:17:00.482186: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [31]:
train_df = pd.DataFrame({'arabic': arabic_train, 'english': english_train})

#train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=42)
val_data = pd.DataFrame({'arabic': arabic_eval, 'english': english_eval})

In [32]:

def tokenize_function(examples):
    inputs = tokenizer(examples['arabic'], padding="max_length", truncation=True, max_length=128)
    outputs = tokenizer(examples['english'], padding="max_length", truncation=True, max_length=128)
    return {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask,
        'labels': outputs.input_ids,
    }

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_data)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7869 [00:00<?, ? examples/s]

Map:   0%|          | 0/775 [00:00<?, ? examples/s]

In [33]:
tokenized_train_dataset

Dataset({
    features: ['arabic', 'english', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 7869
})

In [34]:
train_dataset.shape,val_dataset.shape

((7869, 3), (775, 3))

In [35]:
train_dataset

Dataset({
    features: ['arabic', 'english', '__index_level_0__'],
    num_rows: 7869
})

In [36]:
# Training arguments
from transformers import EarlyStoppingCallback 

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir="./output",
    logging_dir="./logs",
    logging_steps=100,
    eval_steps=100,
    save_steps=500,
    num_train_epochs=4,
    overwrite_output_dir=True,
    save_total_limit=5,
    load_best_model_at_end=True,
    max_steps=1500,
    save_strategy = "steps",
     hub_token =tokenizer
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: ragabhassan716 (translations_smsm). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
100,3.771800,0.405851
200,0.313000,0.222464
300,0.264700,0.200202
400,0.245400,0.189653
500,0.220000,0.185825
600,0.163900,0.179797
700,0.152800,0.176735
800,0.145700,0.173822
900,0.148300,0.170842
1000,0.147700,0.170908


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200

TrainOutput(global_step=1500, training_loss=0.4064720147450765, metrics={'train_runtime': 2528.5038, 'train_samples_per_second': 9.492, 'train_steps_per_second': 0.593, 'total_flos': 6498875813658624.0, 'train_loss': 0.4064720147450765, 'epoch': 3.05})

In [ ]:
'''from transformers import EarlyStoppingCallback

# Training arguments
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir="./output",
    logging_dir="./logs",
    logging_steps=100,
    save_steps=1000,
    num_train_epochs=3,
    overwrite_output_dir=True,
    save_total_limit=3,
    save_strategy="no",
    hub_token =tokenizer
)


# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

trainer.train()'''


In [ ]:
trainer

In [ ]:
stop

In [44]:
from huggingface_hub import notebook_login
notebook_login()

In [46]:
model.save_pretrained("m2m")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


In [47]:
tokenizer.save_pretrained("token")

('token/tokenizer_config.json',
 'token/special_tokens_map.json',
 'token/vocab.json',
 'token/sentencepiece.bpe.model',
 'token/added_tokens.json')

In [45]:
trainer.push_to_hub("without_configration")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


training_args.bin:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ragab167/output/commit/47b0b80f59925141b6960e619e7bd107a49819a1', commit_message='without_configration', commit_description='', oid='47b0b80f59925141b6960e619e7bd107a49819a1', pr_url=None, pr_revision=None, pr_num=None)

In [48]:
model.push_to_hub("Ragab167/m2m_translation_v2")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ragab167/m2m_translation_v2/commit/01a805513ebb326c5e9cb1e9366f58d976abe69a', commit_message='Upload M2M100ForConditionalGeneration', commit_description='', oid='01a805513ebb326c5e9cb1e9366f58d976abe69a', pr_url=None, pr_revision=None, pr_num=None)

In [49]:
tokenizer.push_to_hub("Ragab167/m2m_translation_v2")

CommitInfo(commit_url='https://huggingface.co/Ragab167/m2m_translation_v2/commit/a3231d3d6b37d006468615a602f528e290cbf51f', commit_message='Upload tokenizer', commit_description='', oid='a3231d3d6b37d006468615a602f528e290cbf51f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
df.isna().sum()

In [ ]:
arabic_test.shape

In [51]:
#arabic_test=arabic_test.iloc[120:170]
#english_test=english_test.iloc[120:170]

val_df = pd.DataFrame({'arabic': arabic_test, 'english': english_test})
val_df.head()

,arabic,english
3132,أسطورة ألمانية مُلهِمة لأجيال كتير من الألمان،...,a german legend that inspires many german gene...
1556,وانت يا عم حسن؟ عايز لك كام صرّة؟حسن قال له، ص...,and you hasan how many sachets you wanthasan s...
3071,وأحيانًا، التخلص منه، هو اللي بيخلّي عندنا حاض...,sometimes getting rid of it is what gives us a...
8435,متخيل؟ وانت في الفضاء,imagine and you are in space
4540,باختصار، يا عزيزي، الشخص الوحيد في اليابان الل...,in short my friend the only person in japanwho...


In [52]:
val_df.isna().sum()

arabic     0
english    0
dtype: int64

In [53]:
val_df.shape

(100, 2)

In [54]:
del model1
del tokenized_test_data1

NameError: name 'model1' is not defined

In [55]:
tokenized_test_data = tokenizer(val_df["arabic"].tolist(), return_tensors="pt", padding=True, max_length=128,truncation=True)

In [56]:
tokenized_test_data1=tokenized_test_data.to("cpu")
model1=model.to("cpu")

In [ ]:
generated_translations = model1.generate(**tokenized_test_data1)

#generated_translations = model.generate(input_ids=input_ids, attention_mask=attention_mask)

In [ ]:
decoded_translations = tokenizer.batch_decode(generated_translations, skip_special_tokens=True)



In [ ]:
reference_translations = val_df["english"]

bleu_score = sacrebleu.corpus_bleu(decoded_translations, [reference_translations.tolist()]).score

# Print BLEU score
print("BLEU Score:", bleu_score)

# human judgment

In [ ]:
sentence=' احساس الوحدة احساس صعب جدا عامل زي ما تكون سمكة صغيرة في بحر كبير وغويط ملوش نهاية'

In [ ]:
token_sentence=tokenizer(sentence, return_tensors="pt", padding=True, max_length=128,truncation=True)

In [ ]:
translations_generated = model.generate(**token_sentence)


In [ ]:
translations = tokenizer.batch_decode(translations_generated, skip_special_tokens=True)

In [ ]:
translations